In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [109]:
class exDenseNet(nn.Module):
    def __init__(self, indim, numhidden, outdim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3)
        self.l1 = nn.Linear(indim, numhidden)
        self.l2 = nn.Linear(numhidden, numhidden)
        self.l3 = nn.Linear(numhidden, outdim)
        self.pr1 = nn.PReLU()
        self.pr2 = nn.PReLU()
        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.gap(x)
        print(x.shape)
        x = torch.flatten(x, start_dim=1)
        x = self.l1(x)
        x = self.pr1(x)
        x = self.l2(x)
        x = self.pr2(x)
        x = self.l3(x)
        return F.softmax(x, dim=-1)

In [110]:
model = exDenseNet(64, 256, 10)

In [111]:
from torchsummary import summary
summary(model, (1, 28, 28))

torch.Size([2, 64, 1, 1])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 26, 26]             640
 AdaptiveAvgPool2d-2             [-1, 64, 1, 1]               0
            Linear-3                  [-1, 256]          16,640
             PReLU-4                  [-1, 256]               1
            Linear-5                  [-1, 256]          65,792
             PReLU-6                  [-1, 256]               1
            Linear-7                   [-1, 10]           2,570
Total params: 85,644
Trainable params: 85,644
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.34
Params size (MB): 0.33
Estimated Total Size (MB): 0.67
----------------------------------------------------------------


In [ ]:
import torchvision
from torchvision.datasets import MNIST

In [ ]:
mnist_train = MNIST("./mnist2", download=True, train=True)
mnist_test = MNIST("./mnist2", download=True, train=False)

In [ ]:
import math
from torchvision import transforms
transforms = transforms.Compose(
[
    transforms.ToTensor()
#     transforms.Resize((784))
])




In [ ]:
traindata, trainlabels = zip(*[(transforms(x[0]), x[1]) for x in mnist_train])
valdata, vallabels = zip(*[(transforms(x[0]), x[1]) for x in mnist_test])

In [ ]:
traindata[0].shape
valdata[0].shape


In [ ]:
torch.cat(trainlabels)

In [ ]:
trainds = TensorDataset(torch.cat(traindata)[:32 * 300], torch.cat([torch.tensor([y]) for y in trainlabels])[:32 * 300])
valds = TensorDataset(torch.cat(valdata)[:32 * 30], torch.cat([torch.tensor([y]) for y in vallabels])[:32 * 30])

In [ ]:
traindl = DataLoader(trainds, batch_size=32, shuffle=False)
valdl = DataLoader(valds, batch_size=32, shuffle=False)

In [112]:
opt = optim.AdamW(model.parameters(), lr=.001)
loss_func = F.cross_entropy
n_epochs = 20

train_losses = []
val_losses = []
val_accs = []

with torch.no_grad():
    valpreds, actual = zip(*[(model(torch.flatten(xvb, start_dim=1)), yvb) for xvb, yvb in valdl])
    val_loss = sum([loss_func(vp, vl) for vp, vl in zip(valpreds, actual)])
    print("val loss for epoch ", i, ":", val_loss)
    valpreds = torch.argmax(torch.cat(valpreds), dim=1)
    actual = torch.cat(actual)
    val_acc = sum(p == a for p,a in zip(valpreds, actual))/len(valpreds)
    print("val accuracy for epoch ", i, ":", val_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

for i in range(n_epochs):
    train_loss = 0
    for xtb, ytb in traindl:
        preds = model(torch.flatten(xtb, start_dim=1))
        loss = loss_func(preds, ytb)
        train_loss += loss.item()
        loss.backward()
        opt.step()
        opt.zero_grad()
    train_losses.append(train_loss)
    print("training loss for epoch", i, ":", train_loss, end=", ")
    with torch.no_grad():
        valpreds, actual = zip(*[(model(torch.flatten(xvb, start_dim=1)), yvb) for xvb, yvb in valdl])
        val_loss = sum([loss_func(vp, vl) for vp, vl in zip(valpreds, actual)])
        print("val loss for epoch ", i, ":", val_loss, end=", ")
        valpreds = torch.argmax(torch.cat(valpreds), dim=1)
        actual = torch.cat(actual)
        val_acc = sum(p == a for p,a in zip(valpreds, actual))/len(valpreds)
        print("val accuracy for epoch ", i, ":", val_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 784]

In [ ]:
del model

In [ ]:
# train_losses
# val_losses
val_accs

In [ ]:
val_accs

In [ ]:
import matplotlib.pyplot as plt
plt.plot(val_losses, color="blue")

In [ ]:
plt.plot(vaold, color="red")
plt.plot(val_accs, color="blue")

In [ ]:
from torch import tensor
vaold = [tensor(0.0833),
 tensor(0.8667),
 tensor(0.8646),
 tensor(0.9031),
 tensor(0.8990),
 tensor(0.8979),
 tensor(0.9125),
 tensor(0.9156),
 tensor(0.9146),
 tensor(0.9198),
 tensor(0.9260),
 tensor(0.9219),
 tensor(0.9125),
 tensor(0.9302),
 tensor(0.9281),
 tensor(0.9198),
 tensor(0.9229),
 tensor(0.9146),
 tensor(0.9271),
 tensor(0.9219),
 tensor(0.9240)]